# Testes

### Imports

In [ ]:
from os.path import join
from os import makedirs
from json import load, dump
from datetime import datetime

from unsloth import FastVisionModel
from tqdm.notebook import tqdm
from PIL import Image

import torch

from scripts.authentication import authenticate_huggingface
from scripts.messages import add_inference_message, format_prompt
from scripts.data import LesionData, DatasetAnalysis
from scripts.test import Test, TestResult, GenerationParameters

import scripts.definitions as defs

/home/erfer/Documents/UFSC/INE5453-33-34_TCC/Code/TCC/experiments/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configurações

In [ ]:
MODEL = defs.BASE_MODEL_NAME
QUANTIZED = False  # Isso é sobreescrito no caso de modelos treinados
TEST_ON_TRAINING = False  # Testa o modelo sobre os dados de treinamento
USE_CACHE = False
TEMPERATURE = 0.005

with open(join(defs.TRAINING_PATH, 'models.json'), 'r', encoding='utf-8') as file:
    models = {model_name: defs.Model(**model) for model_name, model in load(file).items()}

model_stats = models[MODEL]
model_path = ''

if model_stats.local:
    model_path = join(defs.RESULTS_PATH, 'adapter_weights', MODEL)
else:
    model_path = MODEL

quantized = model_stats.quantized if model_stats.quantized is not None else QUANTIZED
prompt_template = ''
prompt_type = model_stats.prompt_type

match prompt_type:
    case defs.PromptType.SIMPLE_CLASSIFICATION:
        prompt_template = defs.SIMPLE_CLASSIFICATION_PROMPT_TEMPLATE
    case defs.PromptType.REPORT:
        prompt_template = defs.REPORT_PROMPT_TEMPLATE
    case _:
        raise ValueError('Invalid model type')

model_version = model_stats.version
model_size = model_stats.size

### Autenticação

In [ ]:
authenticate_huggingface()

### Carregamento do dataset

In [ ]:
with open(join(defs.DATA_PATH, 'stt_data', 'training_dataset.json'), 'r', encoding='utf-8') as file:
    training_dataset = [LesionData(**data) for data in load(file)]

with open(join(defs.DATA_PATH, 'training_dataset_analysis.json'), 'r', encoding='utf-8') as file:
    training_dataset_analysis = DatasetAnalysis(**load(file))

with open(join(defs.DATA_PATH, 'stt_data', 'test_dataset.json'), 'r', encoding='utf-8') as file:
    test_dataset = [LesionData(**data) for data in load(file)]

with open(join(defs.DATA_PATH, 'test_dataset_analysis.json'), 'r', encoding='utf-8') as file:
    test_dataset_analysis = DatasetAnalysis(**load(file))

### Carregamento do modelo

In [ ]:
model, tokenizer = FastVisionModel.from_pretrained(
    model_path,
    load_in_4bit=quantized,
    use_gradient_checkpointing='unsloth'
)

FastVisionModel.for_inference(model)

### Preparação do teste

In [4]:
format_prompt(defs.REPORT_PROMPT_TEMPLATE, defs.PromptType.REPORT, training_dataset_analysis)

'Classifique a lesão de pele na imagem, informando a lesão elementar, lesão secundária, coloração, morfologia, tamanho em centímetros, classificação da lesão e classificação de risco.\nPor fim, inclua uma breve conclusão sobre o diganóstico.\nAs opções de classificação de lesões elementares são: Pápula, Placa, Ausente, Mácula/mancha, Nódulo, Pústula, Telangectasias, Comedão, Púrpura, Úlcera, Cisto, Bolha, Equimose, Tumor, Urtica/ponfo, Petéquia, Vesícula.\nAs opções de classificação de lesões secundárias são: Nenhuma, Ceratose, Escamas, Crostas, Exulceração, Escoriação, Liquenificação, Erosão, Cicatriz, Atrofia, Alopécia, Fissura.\nAs opções de classificação de coloração são: Eritematosa (avermelhada), Castanha, Eucrômica, Hipo/acrômica (despigmentada), Amarelada, Violácea, Negra, Perlácea, Azulada.\nAs opções de classificação de morfologia são: Circular ou Arredondada, Irregular/assimétrica, Papilomatosa / Verrucosa, Séssil / Pedunculada, Policíclica, Puntiforme, Lenticular, Folicular

In [ ]:
formatted_prompt = format_prompt(prompt_template, prompt_type, training_dataset_analysis)
messages = add_inference_message(formatted_prompt)

test_name = f'{MODEL}_test_{datetime.now().isoformat()}'.strip('unsloth/')

test = Test(
    tested_model=MODEL.strip('unsloth/'),
    model=model_stats,
    generation_parameters=GenerationParameters(
        max_new_tokens=defs.MAX_TOKENS,
        use_cache=USE_CACHE,
        temperature=TEMPERATURE
    ),
    start_time=datetime.now(),
    end_time=None,
    results_on_test_data=[],
    results_on_training_data=[],
)

tests_path = join(defs.RESULTS_PATH, 'tests')

makedirs(tests_path, exist_ok=True)

### Testes sobre os dados de teste

In [ ]:
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

for idx, exam in enumerate(tqdm(test_dataset, desc='Testing on test data: ')):
    image_path = join(defs.DATA_PATH, 'stt_raw_data', 'dataset', 'images', exam.image)
    image = Image.open(image_path).convert('RGB')

    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors='pt',
    ).to('cuda')

    outputs = model.generate(
        **inputs,
        max_new_tokens=defs.MAX_TOKENS,
        use_cache=USE_CACHE,
        temperature=TEMPERATURE
    )

    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    assistant_message = output.split('assistant')[-1].strip()
    result = TestResult(
        exam_id=exam.exam_id,
        image=exam.image,
        answer=assistant_message
    )

    test.results_on_test_data.append(result)

### Testes sobre os dados de treinamento

In [ ]:
if TEST_ON_TRAINING:
    for idx, exam in enumerate(tqdm(training_dataset, desc='Testing on training data: ')):
        image_path = join(defs.DATA_PATH, 'stt_raw_data', 'dataset', 'images', exam.image)
        image = Image.open(image_path).convert('RGB')

        inputs = tokenizer(
            image,
            input_text,
            add_special_tokens=False,
            return_tensors='pt',
        ).to('cuda')

        outputs = model.generate(
            **inputs,
            max_new_tokens=defs.MAX_TOKENS,
            use_cache=USE_CACHE,
            temperature=TEMPERATURE
        )

        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        assistant_message = result.split('assistant')[-1].strip()
        result = TestResult(
            exam_id=exam.exam_id,
            image=exam.image,
            answer=assistant_message
        )

        test.results_on_training_data.append(result)

### Salvamento do teste

In [ ]:
test.end_time = datetime.now()

test_path = join(tests_path, f'{test_name}.json')

with open(test_path, 'w+', encoding='utf-8') as file:
    dump(test.model_dump(), file, indent=4, ensure_ascii=False)